In [1]:
''' setting before run. every notebook should include this code. '''
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import sys

_r = os.getcwd().split('/')
_p = '/'.join(_r[:_r.index('gate-decorator-pruning')+1])
print('Change dir from %s to %s' % (os.getcwd(), _p))
os.chdir(_p)
sys.path.append(_p)

from config import parse_from_dict
parse_from_dict({
    "base": {
        "task_name": "resnet56_finetune",
        "cuda": True,
        "seed": 1,
        "checkpoint_path": "",
        "epoch": 0,
        "multi_gpus": True,
        "fp16": False
    },
    "model": {
        "name": "cifar.resnet56",
        "num_class": 10,
        "pretrained": False
    },
    "train": {
        "trainer": "normal",
        "max_epoch": 160,
        "optim": "sgd",
        "steplr": [
            [80, 0.1],
            [120, 0.01],
            [160, 0.001]
        ],
        "weight_decay": 5e-4,
        "momentum": 0.9,
        "nesterov": False
    },
    "data": {
        "type": "cifar10",
        "shuffle": True,
        "batch_size": 64,
        "test_batch_size": 128,
        "num_workers": 8
    },
    "loss": {
        "criterion": "softmax"
    },
    "gbn": {
        "finetune_epoch": 40,
        "lr_min": 1e-3,
        "lr_max": 1e-2
    }
})
from config import cfg

Change dir from /root/code/gate-decorator-pruning/run/resnet-56 to /root/code/gate-decorator-pruning
Parsing config file...
** Assert in demo mode. **


usage: ipykernel_launcher.py [-h] [--config CONFIG]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-5c91900f-5696-47bc-a457-c038486324d5.json


In [2]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim

from logger import logger
from main import set_seeds, recover_pack, adjust_learning_rate, _step_lr, _sgdr
from models import get_model
from utils import dotdict

from prune.universal import Meltable, GatedBatchNorm2d, Conv2dObserver, IterRecoverFramework, FinalLinearObserver
from prune.utils import analyse_model, finetune

In [3]:
set_seeds()
pack = recover_pack()

==> Preparing Cifar10 data..
Files already downloaded and verified
Files already downloaded and verified


/root/code/gate-decorator-pruning/models/cifar/resnet56.py:34: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)


In [4]:
GBNs = GatedBatchNorm2d.transform(pack.net)
for gbn in GBNs:
    gbn.extract_from_bn()

-------

#### 70% flops reduced

In [5]:
model_dict = torch.load('./ckps/resnet56_cifar10_70percent_flops_reduced.ckp', map_location='cpu' if not cfg.base.cuda else 'cuda')
pack.net.module.load_state_dict(model_dict)

In [6]:
_ = Conv2dObserver.transform(pack.net.module)
pack.net.module.linear = FinalLinearObserver(pack.net.module.linear)
Meltable.observe(pack, 0.001)
Meltable.melt_all(pack.net)

In [7]:
pack.optimizer = optim.SGD(
    pack.net.parameters(),
    lr=1,
    momentum=cfg.train.momentum,
    weight_decay=cfg.train.weight_decay,
    nesterov=cfg.train.nesterov
)

In [8]:
pack.trainer.test(pack)

{'test_loss': 0.2779307614398908, 'acc@1': 91.79193037974683}

In [9]:
_ = finetune(pack, lr_min=cfg.gbn.lr_min, lr_max=cfg.gbn.lr_max, T=cfg.gbn.finetune_epoch)

  0%|                                                                       | 0/782 [00:00<?, ?it/s]

{'train_loss': 0.0859906351446267, 'epoch_time': 38.19228506088257, 'test_loss': 0.26257172123163564, 'acc@1': 92.51384493670886, 'LR': 0.0014494245524296675}


  0%|                                                                       | 0/782 [00:00<?, ?it/s]

{'train_loss': 0.08416774630775233, 'epoch_time': 38.19319152832031, 'test_loss': 0.2739955025571811, 'acc@1': 92.16772151898734, 'LR': 0.0018994245524296676}


  0%|                                                                       | 0/782 [00:00<?, ?it/s]

...

{'train_loss': 0.030563229463918283, 'epoch_time': 39.35120964050293, 'test_loss': 0.29495988408975965, 'acc@1': 92.83030063291139, 'LR': 0.0019005754475703326}


  0%|                                                                       | 0/782 [00:00<?, ?it/s]

{'train_loss': 0.02765191834696266, 'epoch_time': 38.081024408340454, 'test_loss': 0.2885598743075057, 'acc@1': 92.88963607594937, 'LR': 0.001450575447570333}


100%|█████████████████████████████████████████████████████████████| 782/782 [00:38<00:00, 20.33it/s]


{'train_loss': 0.028634040556905216, 'epoch_time': 38.46471190452576, 'test_loss': 0.29290160272694843, 'acc@1': 92.84018987341773, 'LR': 0.0010005754475703327}


In [10]:
_ = finetune(pack, lr_min=cfg.gbn.lr_min, lr_max=cfg.gbn.lr_min, T=10)

  0%|                                                                       | 0/782 [00:00<?, ?it/s]

{'train_loss': 0.028377088579966132, 'epoch_time': 37.85368323326111, 'test_loss': 0.2879510103147241, 'acc@1': 92.97863924050633, 'LR': 0.001}


  0%|                                                                       | 0/782 [00:00<?, ?it/s]

{'train_loss': 0.02782191441911261, 'epoch_time': 37.820202112197876, 'test_loss': 0.2841554020024553, 'acc@1': 93.09731012658227, 'LR': 0.001}


  0%|                                                                       | 0/782 [00:00<?, ?it/s]

...

{'train_loss': 0.025754430719539333, 'epoch_time': 37.92336654663086, 'test_loss': 0.2933901181515259, 'acc@1': 92.88963607594937, 'LR': 0.001}


  0%|                                                                       | 0/782 [00:00<?, ?it/s]

{'train_loss': 0.025038552658675273, 'epoch_time': 38.24193525314331, 'test_loss': 0.29054723557414885, 'acc@1': 93.09731012658227, 'LR': 0.001}


100%|█████████████████████████████████████████████████████████████| 782/782 [00:39<00:00, 19.80it/s]


{'train_loss': 0.024549294720449106, 'epoch_time': 39.50026035308838, 'test_loss': 0.28829607669311236, 'acc@1': 93.15664556962025, 'LR': 0.001}


In [10]:
pack.net

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(12, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(1, 16, kernel_size=(